In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dropout,Layer
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
drive.mount('/content/drive')

In [ ]:
import pickle
with open('/content/drive/MyDrive/new_train_X.pkl', 'rb') as f:
    new_train_X=pickle.load(f)
with open('/content/drive/MyDrive/new_train_y.pkl', 'rb') as f:
    new_train_y=pickle.load(f)
with open('/content/drive/MyDrive/new_val_X.pkl', 'rb') as f:
    new_val_X=pickle.load(f)
with open('/content/drive/MyDrive/new_val_y.pkl', 'rb') as f:
    new_val_y=pickle.load(f)
with open('/content/drive/MyDrive/dnn_train.pkl', 'rb') as f:
    dnn_train=pickle.load(f)
weather_data = pd.read_csv('/content/drive/MyDrive/weather.csv',encoding='cp949')

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
class TemporalAttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(TemporalAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        super(TemporalAttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W))
        alpha = K.softmax(e, axis=1)
        # 변경: 원래 시퀀스와 가중치를 곱하여 출력 형태를 유지
        context = x * alpha
        return context

    def compute_output_shape(self, input_shape):
        return input_shape


In [ ]:
from tensorflow.keras.utils import plot_model

# 모델 구성을 위한 입력 정의
input_layer = Input(shape=(24, 24))

# 양방향 LSTM 레이어
x = Bidirectional(LSTM(512, return_sequences=True, activation='tanh'))(input_layer)
x = Bidirectional(LSTM(256, return_sequences=True, activation='tanh'))(x)
attention_out = TemporalAttentionLayer()(x)
x = Bidirectional(LSTM(64, activation='tanh'))(attention_out)

# 완전 연결 레이어
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

# 출력 레이어
output_layer = Dense(16)(x)

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse',)
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

model_his = model.fit(
    new_train_X,
    new_train_y,
    callbacks=[early_stopping, checkpoint],
    validation_data=(new_val_X, new_val_y),
    epochs=300,
    batch_size=32,
    shuffle=False
)

In [ ]:
plt.figure(figsize=(24, 6))
plt.plot(model_his.history['loss'][1:], label='Training_loss')
plt.plot(model_his.history['val_loss'][1:], label='Validation_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('Loss.png')
plt.show()

In [ ]:
from keras.models import load_model

model = load_model('/content/best_model.h5', custom_objects={'TemporalAttentionLayer': TemporalAttentionLayer})
loss = model.evaluate(new_val_X, new_val_y)
loss

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
